In [122]:
import requests
from bs4 import BeautifulSoup


base_url="https://pokemondb.net/pokedex/all"
req_all=requests.get(base_url)
cont_all=req_all.content

soup_all=BeautifulSoup(cont_all,"html.parser")
all=soup_all.find_all("table",{"id":"pokedex"})[0].find_all("tr")

pokedex=[]

for item in all:    
    dict={}
    poke_names=item.find_all("a",{"class":"ent-name"})
    if poke_names:
        dict["Pokemon"]=poke_names[0].text
        
    poke_id=item.find_all("span",{"class":"infocard-cell-data"})
    if poke_id:
        dict["Ndex"]=poke_id[0].text
    poke_type=item.find_all("a",{"class":"type-icon"})
    if poke_type:
        init_itr=True
        for i in poke_type:
            if init_itr:
                  typing=i.text
            else :
                typing=typing+"/"+i.text
            init_itr=False   
        dict["Type"]=typing
    poke_total=item.find_all("td",{"class":"cell-total"})
    if poke_total:
        dict["Total"]=poke_total[0].text
        
    poke_stats=item.find_all("td",{"class":"cell-num"})
    if poke_stats:
        dict["Hp"]=poke_stats[1].text
        dict["Attack"]=poke_stats[2].text
        dict["Defense"]=poke_stats[3].text
        dict["Special Attack"]=poke_stats[4].text    
        dict["Special Defense"]=poke_stats[5].text
        dict["Speed"]=poke_stats[6].text
    
    pokedex.append(dict)
    

[{},
 {'Pokemon': 'Bulbasaur',
  'Ndex': '001',
  'Type': 'Grass/Poison',
  'Total': '318',
  'Hp': '45',
  'Attack': '49',
  'Defense': '49',
  'Special Attack': '65',
  'Special Defense': '65',
  'Speed': '45'},
 {'Pokemon': 'Ivysaur',
  'Ndex': '002',
  'Type': 'Grass/Poison',
  'Total': '405',
  'Hp': '60',
  'Attack': '62',
  'Defense': '63',
  'Special Attack': '80',
  'Special Defense': '80',
  'Speed': '60'},
 {'Pokemon': 'Venusaur',
  'Ndex': '003',
  'Type': 'Grass/Poison',
  'Total': '525',
  'Hp': '80',
  'Attack': '82',
  'Defense': '83',
  'Special Attack': '100',
  'Special Defense': '100',
  'Speed': '80'},
 {'Pokemon': 'Venusaur',
  'Ndex': '003',
  'Type': 'Grass/Poison',
  'Total': '625',
  'Hp': '80',
  'Attack': '100',
  'Defense': '123',
  'Special Attack': '122',
  'Special Defense': '120',
  'Speed': '80'},
 {'Pokemon': 'Charmander',
  'Ndex': '004',
  'Type': 'Fire',
  'Total': '309',
  'Hp': '39',
  'Attack': '52',
  'Defense': '43',
  'Special Attack': '60',


In [138]:
import pandas as pd

df=pd.DataFrame(pokedex)
df=df[df["Ndex"].notna()]

df=df.set_index('Ndex')
df1=df.drop_duplicates()
df1.to_csv("Pokemon.csv")
df1.to_json("Pokemon.json",orient="split")